In [ ]:
import os,sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv

In [ ]:
cd ..

In [ ]:
from utils.extract_fred_data import extract_fred_data

In [ ]:
load_dotenv()  # take environment variables from .env.
fred_api_key = os.getenv('fred_api_key')

In [ ]:
fed_funds_rate = extract_fred_data(fred_api_key,'FEDFUNDS')[0]

In [ ]:
fed_funds_rate

In [ ]:
fed_funds_rate.plot()

In [ ]:
fed_funds_rate = fed_funds_rate.round(2)

In [ ]:
number_of_months = int(12*2.1)

fed_funds_rate_rolling_max = fed_funds_rate.rolling(number_of_months, min_periods=1).max()

fed_funds_rate_reversed = fed_funds_rate.iloc[::-1]
fed_funds_rate_rolling_max_forward_reversed = fed_funds_rate_reversed.rolling(number_of_months, min_periods=1).max()

fed_funds_rate_rolling_max_forward = fed_funds_rate_rolling_max_forward_reversed.iloc[::-1]

fed_funds_rate_peaks = fed_funds_rate[(fed_funds_rate == fed_funds_rate_rolling_max) & (fed_funds_rate == fed_funds_rate_rolling_max_forward)]

In [ ]:
fed_funds_rate_peaks = pd.DataFrame(fed_funds_rate_peaks)
fed_funds_rate_peaks.columns = ['peaks']
fed_funds_rate_peaks['peaks'] = fed_funds_rate_peaks['peaks'].astype('float64')

fed_funds_rate_peaks = fed_funds_rate_peaks[fed_funds_rate_peaks['peaks'].notnull()]
fed_funds_rate_peaks

In [ ]:
mask = pd.Series([False] * len(fed_funds_rate_peaks), index=fed_funds_rate_peaks.index)

for i in range(len(fed_funds_rate_peaks)):
    if i == 0 or fed_funds_rate_peaks['peaks'].iloc[i] != fed_funds_rate_peaks['peaks'].iloc[i-1]:
        mask.iloc[i] = True

result_df = fed_funds_rate_peaks[mask]

result_df

In [ ]:
fed_funds_rate_combined = fed_funds_rate.merge(result_df, how='left', left_index=True, right_index=True)
fed_funds_rate_combined

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(12,6))
plt.plot(fed_funds_rate_combined['fedfunds'], label='Federal Funds Rate')
plt.scatter(fed_funds_rate_combined.index, fed_funds_rate_combined['peaks'], color='r', label='Peaks')
plt.fill_between(fed_funds_rate_combined.index, 0, fed_funds_rate_combined['peaks'], color='r', alpha=0.3)
plt.legend()
plt.show()


In [ ]:
fed_funds_rate_combined

In [ ]:
df = fed_funds_rate_combined.copy()

percentages = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4]
results_df = pd.DataFrame()

peak_dates = []

for i in range(len(df)):
    if not pd.isna(df['peaks'].iloc[i]):
        peak_dates.append(df.index[i])

for percentage in percentages:
    current_results = []
    for i in range(len(df)):
        if not pd.isna(df['peaks'].iloc[i]):
            peak_value = df['peaks'].iloc[i]
            peak_date = df.index[i]
            for j in range(i+1, len(df)):
                if df['fedfunds'].iloc[j] <= peak_value * percentage:
                    months_after = (df.index[j].year - peak_date.year) * 12 + df.index[j].month - peak_date.month
                    current_results.append(months_after)
                    break  
                elif j == len(df) - 1:
                    current_results.append(None)

    current_results_df = pd.DataFrame(current_results, columns=[f'Months for rates to fall {-(percentage-1)*100:.0f}% from peak'])
    results_df = pd.concat([results_df, current_results_df], axis=1)

results_df['Peak Date'] = peak_dates

results_df.set_index('Peak Date', inplace=True)

results_df

results_df.dropna(inplace=True)

In [ ]:
results_df.index = results_df.index.strftime('%b %Y')
results_df = results_df.astype('int')
results_df

In [ ]:
# Set the aesthetic style of the plots
sns.set_style("darkgrid")

plt.figure(figsize=(10, 6))

reversed_df = results_df.set_index(results_df.index).T.iloc[::-1]

sns.heatmap(reversed_df, annot=True, fmt=".0f", cmap="YlOrRd")

plt.title('Months After Peak to Reach Specified Percentage of Peak Value')
plt.xlabel('Peak Date')
plt.ylabel('Percentage of Peak Value')

plt.show()

In [ ]:
mean_df = pd.DataFrame(results_df.mean(), columns=['mean'])
std_df = pd.DataFrame(results_df.std(), columns=['std'])
median_std_df = pd.concat([mean_df, std_df], axis=1)
median_std_df

In [ ]:
y = median_std_df.index

x = median_std_df['mean']

xerr = median_std_df['std']
x_upper = x + xerr
x_lower = x - xerr

plt.figure(figsize=(10,6))

plt.plot(x, y, label='Mean', color='blue')

plt.fill_betweenx(y, x_lower, x_upper, color='blue', alpha=0.2, label='1 Std Dev')

plt.title('Mean with Standard Deviation')
plt.ylabel('Percentage of Peak Value')
plt.xlabel('Months After Peak')

plt.legend()

plt.gca().invert_yaxis()

plt.show()

In [ ]:
data = {
    'MEETING DATE': ['11/1/2023', '12/13/2023', '1/31/2024', '3/20/2024', '5/1/2024', '6/12/2024', '7/31/2024', '9/18/2024', '11/7/2024', '12/18/2024'],
    '350-375': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2],
    '375-400': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 2.3],
    '400-425': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 4.4, 9.7],
    '425-450': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 7.3, 15.6, 21.6],
    '450-475': [0.0, 0.0, 0.0, 0.0, 0.1, 2.5, 11.8, 22.5, 28.2, 28.4],
    '475-500': [1.1, 0.0, 0.0, 0.1, 5.3, 18.5, 30.6, 32.9, 28.5, 22.9],
    '500-525': [98.9, 0.9, 0.8, 12.7, 33.6, 39.5, 34.6, 24.9, 16.6, 11.2],
    '525-550': [0.0, 79.6, 70.7, 63.1, 46.2, 31.1, 17.6, 9.7, 5.3, 3.2],
    '550-575': [0.0, 19.5, 26.3, 22.2, 13.8, 7.9, 3.6, 1.7, 0.8, 0.5],
    '575-600': [0.0, 0.0, 2.2, 1.8, 1.1, 0.6, 0.2, 0.1, 0.0, 0.0]
}

rate_probabilities_df = pd.DataFrame(data)

rate_probabilities_df

rate_probabilities_df['MEETING DATE'] = pd.to_datetime(rate_probabilities_df['MEETING DATE'])

rate_probabilities_df['MONTHS FROM NOW'] = rate_probabilities_df['MEETING DATE'].apply(lambda x: (x.year - datetime.now().year) * 12 + x.month - datetime.now().month)

rate_probabilities_df

In [ ]:
df = rate_probabilities_df.copy()

df_numeric = df.iloc[:, 1:-1].apply(pd.to_numeric)

result_data = []

for idx, row in df.iterrows():
    rate_range = df_numeric.loc[idx].idxmax()
    
    lower, upper = map(float, rate_range.split('-'))
    most_likely_rate = (lower + upper) / 2 / 100 
    
    months_from_now = row['MONTHS FROM NOW']
    
    result_data.append({
        'MONTHS FROM NOW': months_from_now,
        'MOST LIKELY RATE': most_likely_rate
    })

result_df = pd.DataFrame(result_data)

In [ ]:
current_rate = fed_funds_rate_combined.loc[fed_funds_rate_combined.index.max(), 'fedfunds']

plt.figure(figsize=(10,6))
plt.title('Path Down from Current Rate based on Historical Medians')
plt.xlabel('Months from Now')
plt.ylabel('Federal Funds Rate (%)')

for i, row in median_std_df.iterrows():
    percentage_label = i.split(' ')[-3]
    print(percentage_label)
    percentage = float(percentage_label.replace('%', '')) / 100
    median_months = row['mean']
    x = np.arange(0, int(median_months) + 1) 
    y = np.linspace(current_rate, current_rate * (1-percentage), len(x))
    plt.plot(x, y, label=f'{percentage_label} of current rate')

plt.plot(result_df['MONTHS FROM NOW'], result_df['MOST LIKELY RATE'], marker='o', linestyle='-', color='red', label='Most likely rate according to FedWatch')

plt.legend()

plt.show()